# Setup Verification Notebook

## Production LLM Deployment: Risk Characterization Before Failure

Run this notebook to verify your environment is correctly configured for the course.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Javihaus/Production_LLM_Deployment/blob/main/setup/04_verify_setup.ipynb)

## Step 1: Install Required Packages

In [ ]:
# Install required packages
!pip install -q anthropic langchain langgraph langchain-anthropic langsmith
!pip install -q numpy pandas scipy matplotlib seaborn statsmodels
!pip install -q tqdm

print("Packages installed successfully!")

## Step 2: Verify Package Imports

In [ ]:
import sys
print(f"Python Version: {sys.version}")
print("="*50)

# Core LLM packages
packages_to_check = [
    ('anthropic', 'Anthropic SDK'),
    ('langchain', 'LangChain'),
    ('langgraph', 'LangGraph'),
    ('numpy', 'NumPy'),
    ('pandas', 'Pandas'),
    ('scipy', 'SciPy'),
    ('matplotlib', 'Matplotlib'),
    ('seaborn', 'Seaborn'),
    ('statsmodels', 'Statsmodels'),
]

all_passed = True
for package, name in packages_to_check:
    try:
        module = __import__(package)
        version = getattr(module, '__version__', 'installed')
        print(f"OK {name}: {version}")
    except ImportError as e:
        print(f"FAIL {name}: Not installed - {e}")
        all_passed = False

print("="*50)
if all_passed:
    print("All packages verified successfully!")
else:
    print("Some packages are missing. Re-run Step 1.")

## Step 3: Configure API Key

### For Google Colab:
1. Click the **key icon** (🔑) in the left sidebar
2. Add a secret named `ANTHROPIC_API_KEY`
3. Paste your API key and enable notebook access

### For Local Environment:
Set the `ANTHROPIC_API_KEY` environment variable

In [ ]:
import os

# Try to get API key from Colab secrets first, then environment
api_key = None

try:
    from google.colab import userdata
    api_key = userdata.get('ANTHROPIC_API_KEY')
    print("API key loaded from Colab secrets")
except:
    api_key = os.environ.get('ANTHROPIC_API_KEY')
    if api_key:
        print("API key loaded from environment variable")

if api_key:
    print(f"API Key: {'*' * 20}...{api_key[-4:]}")
    print(f"Length: {len(api_key)} characters")
else:
    print("WARNING: No API key found!")
    print("Please configure your ANTHROPIC_API_KEY before proceeding.")

## Step 4: Test Anthropic API Connection

In [ ]:
import anthropic

# Initialize client
if api_key:
    client = anthropic.Anthropic(api_key=api_key)

    try:
        # Simple test message
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=100,
            messages=[
                {
                    "role": "user",
                    "content": "Respond with exactly: 'API connection successful! Ready for Production LLM Deployment course.'"
                }
            ]
        )

        print("API Response:")
        print(response.content[0].text)
        print("\n" + "="*50)
        print(f"Model: {response.model}")
        print(f"Input tokens: {response.usage.input_tokens}")
        print(f"Output tokens: {response.usage.output_tokens}")

        # Estimate cost
        input_cost = response.usage.input_tokens * 3 / 1_000_000
        output_cost = response.usage.output_tokens * 15 / 1_000_000
        print(f"Estimated cost: ${input_cost + output_cost:.6f}")
        print("\nAPI test passed!")

    except anthropic.AuthenticationError:
        print("ERROR: Invalid API key. Please check your key and try again.")
    except anthropic.RateLimitError:
        print("ERROR: Rate limit exceeded. Wait a moment and try again.")
    except Exception as e:
        print(f"ERROR: {type(e).__name__}: {e}")
else:
    print("Skipping API test - no API key configured.")

## Step 5: Test LangChain Integration

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage

if api_key:
    try:
        # Initialize LangChain with Claude
        llm = ChatAnthropic(
            model="claude-sonnet-4-5-20250929",
            api_key=api_key,
            max_tokens=100
        )

        # Test invocation
        response = llm.invoke([HumanMessage(content="Say 'LangChain integration working!' only.")])
        print(f"LangChain Response: {response.content}")
        print("\nLangChain test passed!")

    except Exception as e:
        print(f"ERROR: {type(e).__name__}: {e}")
else:
    print("Skipping LangChain test - no API key configured.")

## Step 6: Test Statistical Libraries

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Generate sample data
np.random.seed(42)
group_a = np.random.normal(0.75, 0.1, 30)  # Higher accuracy
group_b = np.random.normal(0.45, 0.15, 30)  # Lower accuracy

# Perform t-test
t_stat, p_value = stats.ttest_ind(group_a, group_b)

print("Statistical Test Results:")
print(f"  Group A mean: {group_a.mean():.3f} (SD: {group_a.std():.3f})")
print(f"  Group B mean: {group_b.mean():.3f} (SD: {group_b.std():.3f})")
print(f"  t-statistic: {t_stat:.3f}")
print(f"  p-value: {p_value:.6f}")
print(f"  Significant difference: {p_value < 0.05}")

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Box plot
data = pd.DataFrame({'Group A': group_a, 'Group B': group_b})
data_melted = data.melt(var_name='Group', value_name='Accuracy')
sns.boxplot(data=data_melted, x='Group', y='Accuracy', ax=axes[0])
axes[0].set_title('Accuracy Distribution by Group')
axes[0].set_ylim(0, 1)

# Distribution plot
sns.histplot(group_a, kde=True, label='Group A', alpha=0.5, ax=axes[1])
sns.histplot(group_b, kde=True, label='Group B', alpha=0.5, ax=axes[1])
axes[1].set_title('Accuracy Distributions')
axes[1].set_xlabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nStatistical libraries test passed!")

## Step 7: Verify Google Drive Access (Optional)

In [ ]:
# Uncomment to mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Create course directory
# !mkdir -p /content/drive/MyDrive/LLM_Course
# print("Course directory created at /content/drive/MyDrive/LLM_Course")

print("Google Drive mounting available (uncomment to enable)")

## Summary

In [ ]:
print("="*60)
print("SETUP VERIFICATION SUMMARY")
print("="*60)

checks = [
    ("Python 3.10+", sys.version_info >= (3, 10)),
    ("Core packages installed", all_passed if 'all_passed' in dir() else False),
    ("API key configured", api_key is not None),
]

for check_name, passed in checks:
    status = "PASS" if passed else "FAIL"
    print(f"  [{status}] {check_name}")

print("="*60)

if all(passed for _, passed in checks):
    print("\nAll checks passed! You're ready for the course.")
    print("\nNext steps:")
    print("  1. Review the course syllabus (SYLLABUS.md)")
    print("  2. Complete pre-course reading")
    print("  3. Identify your deployment scenario for exercises")
    print("  4. Join the course Slack workspace")
else:
    print("\nSome checks failed. Please resolve issues before Session 1.")
    print("\nNeed help? Contact: javier@jmarin.info")

---

## Troubleshooting

### API Key Issues
- Verify key starts with `sk-ant-api03-`
- Check for extra spaces when copying
- Ensure payment method is configured in Anthropic Console

### Package Installation Issues
- Try restarting runtime and re-running Step 1
- For Colab: Runtime > Restart runtime

### Connection Issues
- Check internet connection
- Verify Anthropic API status: https://status.anthropic.com

---

**Course:** Production LLM Deployment: Risk Characterization Before Failure  
**Instructor:** Javier Marín  
**Contact:** javier@jmarin.info